### Some explanations

Конспект: https://drive.google.com/drive/folders/1O7BqXJ3t6xTkB3ppRiqrWISuy5e2qz0R?usp=sharing

Кратко: \
"Традиционные" способы постоения рекомендательных систем основанны на представление рекомендательной метрицы через матрицу "кому рекомендуем" и "что рекомендуем".  Собственно, алгоритмы: \
* ALS        
* SVD      
* NN \
Ансамбль из них даёт результат, превосходящий их по mAE и NDCG. \

Также есть LTR подход. Основное отличие:  
1) "Традиционный" ML решает проблему прогнозирования  для одного экземпляра за раз. Например, почта. Письмо -> фичи -> решение, спам/не спам.  \
2) LTR решает проблему ранжирования в списке предметов. Целью LTR является разработка оптимального порядка этих товаров. Таким образом, LTR не особо заботится о точном балле, который получает каждый предмет, а об относительном его упорядочении среди остальных. \
Представителями LTR являются:
* RankNet
* LambdaRank - улучшили cost func из RankNet, придумав идею с  градиентами
* LambdaMART - основан на деревьях и costFunc из LambdaRank

----
Источники: \
https://towardsdatascience.com/anime-recommendation-engine-from-matrix-factorization-to-learning-to-rank-845d4a9af335 \

https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/MSR-TR-2010-82.pdf \

https://medium.com/@nikhilbd/intuitive-explanation-of-learning-to-rank-and-ranknet-lambdarank-and-lambdamart-fe1e17fac418 \

http://proceedings.mlr.press/v30/Wang13.pdf \

### Prerequirements for colab

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!pip install kaggle

In [0]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [0]:
api_token = {"username":"__nope____","key":"___xe-xe-xe____"}
import json
import zipfile
import os
with open('/content/gdrive/My Drive/МФТИ/Colab Notebooks/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!chmod 600 '/content/gdrive/My Drive/МФТИ/Colab Notebooks/kaggle.json'
!kaggle config path -p '/content/gdrive/My Drive/МФТИ/Colab Notebooks'
!kaggle datasets download -d azathoth42/myanimelist

for file in os.listdir():
    zip_ref = zipfile.ZipFile(file, 'r')
    zip_ref.extractall()
zip_ref.close()

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 11, in <module>
    load_entry_point('kaggle==1.5.1.1', 'console_scripts', 'kaggle')()
  File "/usr/local/lib/python2.7/dist-packages/pkg_resources/__init__.py", line 487, in load_entry_point
    return get_distribution(dist).load_entry_point(group, name)
  File "/usr/local/lib/python2.7/dist-packages/pkg_resources/__init__.py", line 2728, in load_entry_point
    return ep.load()
  File "/usr/local/lib/python2.7/dist-packages/pkg_resources/__init__.py", line 2346, in load
    return self.resolve()
  File "/usr/local/lib/python2.7/dist-packages/pkg_resources/__init__.py", line 2352, in resolve
    module = __import__(self.module_name, fromlist=['__name__'], level=0)
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 116, in authenticate
    self.config_file, self.config

IsADirectoryError: ignored

In [0]:
# 

In [0]:
!kaggle datasets download -d azathoth42/myanimelist

### Some code

Не требовалось писать/запускать код, но посмотреть на полезную библиотеку и методы стоит

In [0]:
!pip install surprise

    100% |████████████████████████████████| 3.3MB 8.8MB/s 
  Running setup.py bdist_wheel for scikit-surprise ... - \ | / - \ | / - \ | / - done
  Stored in directory: /root/.cache/pip/wheels/ec/c0/55/3a28eab06b53c220015063ebbdb81213cd3dcbb72c088251ec
Successfully built scikit-surprise


In [0]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from pandas import Series, DataFrame
import pandas as pd
import re
%pylab inline
import matplotlib.pyplot as plt
from collections import defaultdict

from surprise import SVD
from surprise import Dataset

from surprise import NormalPredictor

from surprise import Reader
from surprise import NMF
from surprise import SVDpp
from surprise import accuracy
from surprise.model_selection import train_test_split
import webbrowser

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.6/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['random', 'copy', 'repeat']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from pandas import Series, DataFrame
import pandas as pd
import re
%pylab inline
import matplotlib.pyplot as plt
from collections import defaultdict

from surprise import SVD
from surprise import Dataset

from surprise import NormalPredictor

from surprise import Reader
from surprise import NMF
from surprise import SVDpp
from surprise import accuracy
from surprise.model_selection import train_test_split

Populating the interactive namespace from numpy and matplotlib


In [0]:
users=pd.read_csv('/content/gdrive/My Drive/МФТИ/Colab Notebooks/Машинка, домашка/hw6/UserAnimeList.csv', nrows=20000)

In [5]:
users

,username,anime_id,my_watched_episodes,my_start_date,my_finish_date,my_score,my_status,my_rewatching,my_rewatching_ep,my_last_updated,my_tags
0,karthiga,21,586,0000-00-00,0000-00-00,9,1,NaN,0,1362307973,NaN
1,karthiga,59,26,0000-00-00,0000-00-00,7,2,NaN,0,1362923691,NaN
2,karthiga,74,26,0000-00-00,0000-00-00,7,2,NaN,0,1367081015,NaN
3,karthiga,120,26,0000-00-00,0000-00-00,7,2,NaN,0,1362308037,NaN
4,karthiga,178,26,0000-00-00,0000-00-00,7,2,0.0,0,1364399953,NaN
5,karthiga,210,161,0000-00-00,0000-00-00,7,2,NaN,0,1362923826,NaN
6,karthiga,232,70,0000-00-00,0000-00-00,6,2,NaN,0,1362849882,NaN
7,karthiga,233,78,0000-00-00,0000-00-00,6,2,NaN,0,1362893384,NaN
8,karthiga,249,167,0000-00-00,0000-00-00,8,2,NaN,0,1363709086,NaN
9,karthiga,269,366,0000-00-00,0000-00-00,10,2,NaN,0,1362303563,NaN


In [0]:


df =users[['username','anime_id','my_score']].iloc[:2000]
testdata=df[df['my_score']==0]
traindata=df[df['my_score']!=0]
reader = Reader(rating_scale=(1, 2000))
readertrain = Reader(rating_scale=(1, traindata.shape[0]))
readertest = Reader(rating_scale=(1, testdata.shape[0]))

data = Dataset.load_from_df(df[['username','anime_id','my_score']], reader)
train_data=Dataset.load_from_df(traindata[['username','anime_id','my_score']], readertrain)
test_data=Dataset.load_from_df(testdata[['username','anime_id','my_score']], readertest)


In [8]:
trainset, testset = train_test_split(train_data, test_size=.25)

# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 1.6496


1.649599739193392

In [9]:
algo = SVDpp()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 1.6345


1.6344681646650594

In [10]:
algo = NMF()
trainset, testset = train_test_split(train_data, test_size=.25)
# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 2.2431


2.243079237941695

In [11]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):


    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


# First train an SVD algorithm on the movielens dataset.
trainset, testset = train_test_split(data, test_size=.25)
#trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
#testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

Damonashu [2904, 4186, 934, 6594, 2236, 335, 22535, 7662, 30276, 1536]
bskai [132, 189, 145, 846, 1735, 7858, 2472, 24, 249, 1690]
shuzzable [16668, 4654, 8937, 918, 15583, 17265, 12413, 9656, 10588, 80]
RedvelvetDaisuki [10863, 23847, 11111, 16592, 6347, 198, 1735, 1604, 7817, 11617]
Bas_G [15809, 28805, 18857, 28999, 3712, 23281, 9253, 14345, 25013, 22535]
karthiga [4814, 10030, 7817, 857, 21, 210, 4722, 74, 3731, 11013]
terune_uzumaki [269, 6347, 479, 74]


In [12]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):


    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


# First train an SVD algorithm on the movielens dataset.
#trainset, testset = train_test_split(train_data, test_size=.25)
trainset = train_data.build_full_trainset()
algo = NMF()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

karthiga [17074, 15117, 10029, 8063, 12355, 6553, 34599, 1, 1535, 801]
RedvelvetDaisuki [189, 21, 801, 966, 191, 190, 4181, 4163, 4059, 28171]
Damonashu [966, 34599, 8063, 17074, 10029, 21881, 13601, 15117, 28171, 23251]
bskai [17074, 801, 22789, 1425, 12531, 35079, 437, 8063, 6553, 35857]
shuzzable [17074, 19, 1535, 1, 801, 10029, 15117, 12355, 34599, 966]
terune_uzumaki [17074, 1, 801, 12355, 5, 10029, 8063, 437, 6553, 6]
Bas_G [17074, 12355, 437, 19, 801, 21, 57, 10080, 35857, 189]


In [13]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):


    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


# First train an SVD algorithm on the movielens dataset.
#trainset, testset = train_test_split(train_data, test_size=.25)
trainset = train_data.build_full_trainset()
algo = SVDpp()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

karthiga [13767, 1535, 10162, 5365, 23283, 5114, 12355, 22789, 268, 431]
RedvelvetDaisuki [21, 801, 471, 90, 5, 7311, 10163, 21881, 22, 6]
Damonashu [7054, 1412, 11123, 10029, 437, 5114, 9926, 4181, 15117, 10408]
bskai [22535, 7724, 30015, 13161, 152, 5630, 13659, 10793, 18677, 5114]
shuzzable [1535, 12355, 1575, 4181, 5114, 5365, 7054, 1735, 1, 1007]
terune_uzumaki [1535, 2904, 12355, 7054, 21, 19, 189, 4181, 3092, 1]
Bas_G [12355, 12531, 21, 17819, 1425, 35857, 1, 19, 34599, 431]


In [14]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):


    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

trainset, testset = train_test_split(test_data, test_size=.99)
# First train an SVD algorithm on the movielens dataset.
#trainset, testset = train_test_split(train_data, test_size=.25)
trainset = train_data.build_full_trainset()
algo = SVD()
algo.fit(trainset)


# Than predict ratings for all pairs (u, i) that are NOT in the training set.
#testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

RedvelvetDaisuki [6573, 11757, 6205, 23273, 11013, 27655, 12189, 1482, 14075, 24]
Bas_G [18857, 433, 4382, 5734, 8142, 6707, 281, 20583, 18679, 1723]
Damonashu [31478, 356, 28805, 71, 66, 8934, 477, 6573, 6594, 3089]
shuzzable [4382, 18679, 6045, 20507, 4896, 3972, 18139, 5681, 10721, 19365]
bskai [7647, 20, 8407, 6773, 1397, 1067, 5277, 3359, 355, 4213]
